# LDA models different decades (paragraphs as input)

In [2]:
#System
import os
import sys
from tqdm import tqdm 

#Data structure manipulation
import pandas as pd
import numpy as np

#text cleaning 
import re
import string

#nlp
import nltk
from nltk.corpus import stopwords
import spacy

#gensim
import gensim
from gensim import corpora
from gensim.test.utils import datapath
# unlist nested lists
from itertools import chain

# count word frequencies
from collections import defaultdict

# vizualization
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

# timer function
import time

## Reading the data

In [3]:
#Path to where the raw text files are stored within the session folders, i.e, converted sessions.
origin_path = "C:/DATA/convSes"

In [4]:
#A function for reading in the speeches
def read_text(file_path, file):
    
    '''Reading the text files'''
    
    with open(file_path, 'r', encoding='utf-8') as file:
        doc=file.read()
    return doc

In [5]:
#Bringing in all the speeches
doc_set = []
for i in range (0,50):
    year = 1970 + i
    session = "session " + str(25+i)+ " - "  + str(year)
    data_path = f"{origin_path}\\{session}"
    os.chdir(data_path)
    for file in os.listdir():
        if file[-4:]=='.txt':
            file_path = f"{data_path}\\{file}"
            doc_set.append({'Year': year, 'ISO_Code': file[:3] , 'text': read_text(file_path, file)})
        else:
            print(file)
            pass

word2vec.model


In [6]:
columns=['Year', 'ISO_Code', 'text']
dataset = pd.concat([pd.DataFrame([i], columns=columns) for i in tqdm(doc_set)], ignore_index=True)
dataset.head()

100%|██████████| 8288/8288 [00:04<00:00, 1837.03it/s]


,Year,ISO_Code,text
0,1970,ALB,33: May I first convey to our President the co...
1,1970,ARG,177.\t : It is a fortunate coincidence that pr...
2,1970,AUS,100.\t It is a pleasure for me to extend to y...
3,1970,AUT,155.\t May I begin by expressing to Ambassado...
4,1970,BEL,"176. No doubt each of us, before coming up to ..."


**Creating subset including only the G20 states**

In [7]:
# only select states belonging to the G20 group (minus south afrika and EU representatives)
g20 =  dataset.ISO_Code.isin(['CAN','FRA', 'DEU', 'USA', 'GBR', 'ITA', 'JPN','ARG', 'Aus', 'BRA', 'IND', 'IDN', 
                        'CAN', 'MEX', 'RUS', 'SAU', 'KOR', 'TUR', 'CHN'])
G20 = dataset[g20]

# reset ascending index for subset dataset
G20.reset_index(inplace = True, drop = True)
G20.head()

,Year,ISO_Code,text
0,1970,ARG,177.\t : It is a fortunate coincidence that pr...
1,1970,BRA,"1.\tMr. President, I should like, first of all..."
2,1970,CAN,The General Assembly is fortunate indeed to ha...
3,1970,FRA,"84.\t Within one month, when we celebrate the..."
4,1970,GBR,"110.\t Mr. President, I should like first to s..."


In [8]:
# testing a random sample of speeches
G20.sample(10)

,Year,ISO_Code,text
486,2000,GBR,I am conscious that\nthe central problem for a...
463,1999,ARG,We\nare pleased that an important figure in Na...
5,1970,IDN,"101.\tOn behalf of the Indonesian delegation, ..."
734,2015,IDN,Let me begin by congratulating Mr. Mogens Lykk...
514,2002,ARG,"﻿First, I wish to congratulate you, Mr. Presid..."
435,1997,FRA,"﻿May I first say, Sir, how pleased my country ..."
672,2011,JPN,I would like to begin \nby congratulating Mr. ...
485,2000,FRA,I have\nthe honour of speaking this year on be...
427,1996,SAU,"﻿It gives me pleasure, as we begin the work of..."
282,1988,BRA,"﻿\nMr. President, a tradition dating back to t..."


**Creating subsets for the 70s,80s,90s,2000s,2010s**

In [9]:
G20_70s = G20[(G20['Year'] >= 1970) & (G20['Year'] < 1980)] 
G20_80s = G20[(G20['Year'] >= 1980) & (G20['Year'] < 1990)]
G20_90s = G20[(G20['Year'] >= 1990) & (G20['Year'] < 2000)]
G20_2000s = G20[(G20['Year'] >= 2000) & (G20['Year'] < 2010)]
G20_2010s = G20[(G20['Year'] >= 2010) & (G20['Year'] < 2020)]

**Split speeches for every decade slice into paragraphs**

In [10]:
#70s

# Split speeches by delimiter \n and...
# ...stack a new row for each paragraph to dataframe... 
# ...with context info from corresponding speech.

temporary_file = G20_70s['text'].str.split('\.\s?\s?\s?\n', expand=True).stack().reset_index(level=0)
temporary_file.rename(columns={'level_0': 'speech_index', 0: 'text'}, inplace=True)
# temporary_file.reset_index().drop('index',1)

# merge context info from speeches with texts of paragraphs
G20_70s = G20_70s.drop('text', 1) # drop original text column
G20_70s = temporary_file.merge(G20_70s, right_index=True, left_on='speech_index', how='outer')

# reorder columns in new datset
G20_70s['paragraph_index'] = np.arange(len(G20_70s))
cols = G20_70s.columns.tolist()
cols = cols[-1:] + cols[:-1]
G20_70s = G20_70s[cols].set_index('paragraph_index')
G20_70s = G20_70s[1:]

In [11]:
#80s

# Split speeches by delimiter \n and...
# ...stack a new row for each paragraph to dataframe... 
# ...with context info from corresponding speech.

temporary_file = G20_80s['text'].str.split('\.\s?\s?\s?\n', expand=True).stack().reset_index(level=0)
temporary_file.rename(columns={'level_0': 'speech_index', 0: 'text'}, inplace=True)
# temporary_file.reset_index().drop('index',1)

# merge context info from speeches with texts of paragraphs
G20_80s = G20_80s.drop('text', 1) # drop original text column
G20_80s = temporary_file.merge(G20_80s, right_index=True, left_on='speech_index', how='outer')

# reorder columns in new datset
G20_80s['paragraph_index'] = np.arange(len(G20_80s))
cols = G20_80s.columns.tolist()
cols = cols[-1:] + cols[:-1]
G20_80s = G20_80s[cols].set_index('paragraph_index')
G20_80s = G20_80s[1:]

In [12]:
#90s

# Split speeches by delimiter \n and...
# ...stack a new row for each paragraph to dataframe... 
# ...with context info from corresponding speech.

temporary_file = G20_90s['text'].str.split('\.\s?\s?\s?\n', expand=True).stack().reset_index(level=0)
temporary_file.rename(columns={'level_0': 'speech_index', 0: 'text'}, inplace=True)
# temporary_file.reset_index().drop('index',1)

# merge context info from speeches with texts of paragraphs
G20_90s = G20_90s.drop('text', 1) # drop original text column
G20_90s = temporary_file.merge(G20_90s, right_index=True, left_on='speech_index', how='outer')

# reorder columns in new datset
G20_90s['paragraph_index'] = np.arange(len(G20_90s))
cols = G20_90s.columns.tolist()
cols = cols[-1:] + cols[:-1]
G20_90s = G20_90s[cols].set_index('paragraph_index')
G20_90s = G20_90s[1:]

In [13]:
#2000s

# Split speeches by delimiter \n and...
# ...stack a new row for each paragraph to dataframe... 
# ...with context info from corresponding speech.

temporary_file = G20_2000s['text'].str.split('\.\s?\s?\s?\n', expand=True).stack().reset_index(level=0)
temporary_file.rename(columns={'level_0': 'speech_index', 0: 'text'}, inplace=True)
# temporary_file.reset_index().drop('index',1)

# merge context info from speeches with texts of paragraphs
G20_2000s = G20_2000s.drop('text', 1) # drop original text column
G20_2000s = temporary_file.merge(G20_2000s, right_index=True, left_on='speech_index', how='outer')

# reorder columns in new datset
G20_2000s['paragraph_index'] = np.arange(len(G20_2000s))
cols = G20_2000s.columns.tolist()
cols = cols[-1:] + cols[:-1]
G20_2000s = G20_2000s[cols].set_index('paragraph_index')
G20_2000s = G20_2000s[1:]

In [14]:
#2010s

# Split speeches by delimiter \n and...
# ...stack a new row for each paragraph to dataframe... 
# ...with context info from corresponding speech.

temporary_file = G20_2010s['text'].str.split('\.\s?\s?\s?\n', expand=True).stack().reset_index(level=0)
temporary_file.rename(columns={'level_0': 'speech_index', 0: 'text'}, inplace=True)
# temporary_file.reset_index().drop('index',1)

# merge context info from speeches with texts of paragraphs
G20_2010s = G20_2010s.drop('text', 1) # drop original text column
G20_2010s = temporary_file.merge(G20_2010s, right_index=True, left_on='speech_index', how='outer')

# reorder columns in new datset
G20_2010s['paragraph_index'] = np.arange(len(G20_2010s))
cols = G20_2010s.columns.tolist()
cols = cols[-1:] + cols[:-1]
G20_2010s = G20_2010s[cols].set_index('paragraph_index')
G20_2010s = G20_2010s[1:]

## Pre-processing

**Defining/loading in the needed functions --> can we do this in a cleaner way??**

In [15]:
nlp = spacy.load('en_core_web_lg')#run in conda to download the library --> python -m download en_core_web_lg 

In [16]:
def init_proc(text, stop_words=[]):
    
    '''Pre-processing the input in single list'''
    
    stops = stopwords.words("english")
    stops.extend(stop_words)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\t', ' ', text)
    text = re.sub(r'\-', ' ', text)
    text = re.sub(r'\s\s+', ' ', text)
    text = re.sub(r'[0-9]+','', text)
    text = re.sub(r'[^\w\s]', '', text)
    lower = text.lower()
    doc = nlp(lower)
    words = []
    for token in doc:
        lemma = token.lemma_
        if lemma not in stops:
                words.append(lemma)
    return words

In [17]:
stop_words =['general', 'assembly', 'conference', 'session', 'congratulations', 
             'congratulate', 'secretarygeneral','members', 'member', 'united', 'nations', 
             'nation', 'statement', 'honour','every', 'sir', 'majesty', 'president', 
             'minister', 'prime', 'ambassador', 'thank', 'thanks', 'world', 'international', 
             'states', 'we', 'us', 'they', 'system', 'organization','say', 'think', 'know', 
             'want', 'need', 'let', 'ask', 'go', 'look', 'stand', 'open', 'give', 'see', 'come', 
             'make', 'made', 'meet','act', 'use', 'take', 'bring', 'ensure', 'able', 'assume', 
             'continue', 'change', 'progress', 'process', 'year', 'years', 'time', 'today',  
             'would', 'will', 'might', 'together', 'common', 'future', 'one', 'order', 'end', 
             'new', 'necessary', 'major', 'minor', 'many', 'people', 'peoples', 'appropriate', 
             'historic', 'adequate', 'best', 'better', 'confident', 'important', 'special',
             'great', 'therefore', 'thus', 'hence', 'like', 'particularly', 'many', 'much', 
             'greater', 'especially', 'towards', 'always', 'whether', 'around',
             'possible', 'clear', 'simply', 'must', 'also', 'however', 'mr',
             'united', 'kingdom', 'great', 'britain', 'france', 'germany','italy', 'japan',
             'canada', 'usa', 'argentina', 'australia', 'china', 'brazil', 'india', 'indonesia',  
             'mexico', 'russia', 'saudi', 'arabia', 'south', 'korea', 'turkey','liechtenstein',
             'I', ' ', '  ']

In [18]:
#loading the bigram models from the bigram_trigram_model_notebooks
#temp_file1 = datapath("C:/DATA/DTM/phrasers/bigram_speeches")
#temp_file2 = datapath("C:/DATA/DTM/phrasers/trigram_speeches")
from gensim.models.phrases import Phrases
bigram_phraser = Phrases.load("C:/DATA/DTM/phrasers/bigram_speeches")
trigam_phraser = Phrases.load("C:/DATA/DTM/phrasers/trigram_speeches")

In [19]:
#Function to preprocess speeches and also detect bi and trigrams
def pre_proc_comb(corpus,stop_words=[]):
    
    '''Looping the pre-processing over a list. Also checks if input is correct'''
    
    l = []
    if isinstance(corpus, str):
        l.append(init_proc(corpus,stop_words))
    elif all(isinstance(s, str) for s in corpus):    
        for item in tqdm(corpus):
            l.append(init_proc(item,stop_words))
    else:
        print("Error: This function only accepts strings or a list of strings.")
    bigram_token = []
    for j in l:
        bigram_token.append(bigram_phraser[j])
    trigram_token = []
    for i in bigram_token:
        trigram_token.append(trigam_phraser[i])
    return trigram_token

**Preprocessing the 5 dataslices**

In [20]:
#70s
text_corpus_70s = G20_70s['text'].values.tolist()
processed_corpus_70s = pre_proc_comb(text_corpus_70s,stop_words)

100%|██████████| 7730/7730 [02:27<00:00, 52.46it/s]


In [21]:
#80s
text_corpus_80s = G20_80s['text'].values.tolist()
processed_corpus_80s = pre_proc_comb(text_corpus_80s,stop_words)

100%|██████████| 8461/8461 [02:47<00:00, 50.60it/s]


In [22]:
#90s
text_corpus_90s = G20_90s['text'].values.tolist()
processed_corpus_90s = pre_proc_comb(text_corpus_90s,stop_words)

100%|██████████| 8020/8020 [02:02<00:00, 65.71it/s]


In [23]:
#2000s
text_corpus_2000s = G20_2000s['text'].values.tolist()
processed_corpus_2000s = pre_proc_comb(text_corpus_2000s,stop_words)

100%|██████████| 5680/5680 [01:18<00:00, 71.92it/s]


In [24]:
#2010s
text_corpus_2010s = G20_2010s['text'].values.tolist()
processed_corpus_2010s = pre_proc_comb(text_corpus_2010s,stop_words)

100%|██████████| 5208/5208 [01:28<00:00, 58.96it/s]


## Building the LDA models for every decade sice 

### 70s

In [25]:
%%time
# Placing the processed corpus in necessary format for LDA

dictionary_70s = corpora.Dictionary(processed_corpus_70s)
doc_term_matrix_70s = [dictionary_70s.doc2bow(speech) for speech in processed_corpus_70s]

# Creating the LDA object 

LDA = gensim.models.ldamodel.LdaModel

#building the model
lda_model_70s = LDA(corpus = doc_term_matrix_70s, id2word = dictionary_70s, num_topics = 10, random_state=100,
               chunksize= 1000, passes = 1000, iterations = 500)

Wall time: 39min 59s


In [26]:
#Saving the model locally 
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/70s")
lda_model_70s.save(temp_file)

In [27]:
#Loading the model if it is already built
LDA = gensim.models.ldamodel.LdaModel
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/70s")
lda_model_70s = LDA.load(temp_file)

In [28]:
#investigating the word distribution over the different topics
#lda_model_70s.print_topics()

In [29]:
#import pyLDAvis
#import pyLDAvis.gensim_models as gensimvis
#pyLDAvis.enable_notebook()

In [30]:
vis70s = gensimvis.prepare(lda_model_70s,doc_term_matrix_70s,dictionary_70s,mds= 'mmds', R=30)
vis70s

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.214286 -0.083363       1        1  17.850769
4     -0.005352  0.071118       2        1  15.656444
0     -0.139648  0.282163       3        1  12.153195
2      0.129501 -0.126591       4        1  11.778526
7      0.336124 -0.045464       5        1  11.108960
3      0.267394  0.181245       6        1   9.584940
1      0.093818  0.319977       7        1   8.661816
9      0.143026 -0.344416       8        1   6.364758
5     -0.373950  0.128650       9        1   5.235452
8     -0.236627 -0.383319      10        1   1.605140, topic_info=            Term         Freq        Total Category  logprob  loglift
476      nuclear  1305.000000  1305.000000  Default  30.0000  30.0000
28         peace  2207.000000  2207.000000  Default  29.0000  29.0000
714      develop  1359.000000  1359.000000  Default  28.0000  28.0000
715     economic  1605.000000  1605.000000  Default  27.0000  27.0000
119      country  3636.000000  3636.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
1519       paris    32.218171    62.979529  Topic10  -4.8548   3.4617
3334   elsewhere    25.838357    45.981374  Topic10  -5.0755   3.5556
2326         big    30.710595    72.696823  Topic10  -4.9027   3.2703
2579  democratic    33.391361   144.989865  Topic10  -4.8190   2.6636
678      foreign    37.605726   372.108917  Topic10  -4.7002   1.8399

[647 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
75        4  0.984563       access
5195      9  0.972291  accordingly
458       2  0.182571      achieve
458       3  0.497154      achieve
458       4  0.053367      achieve
...     ...       ...          ...
582       3  0.410100          yet
582       5  0.165264          yet
582       8  0.180566          yet
2314      6  0.992425     zimbabwe
5361      6  0.968356      zionist

[1174 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 1, 3, 8, 4, 2, 10, 6, 9])

### 80s

In [31]:
%%time
# Placing the processed corpus in necessary format for LDA

dictionary_80s = corpora.Dictionary(processed_corpus_80s)
doc_term_matrix_80s = [dictionary_80s.doc2bow(speech) for speech in processed_corpus_80s]

# Creating the LDA object 

LDA = gensim.models.ldamodel.LdaModel

#building the model
lda_model_80s = LDA(corpus = doc_term_matrix_80s, id2word = dictionary_80s, num_topics = 10, random_state=100,
               chunksize= 1000, passes = 1000, iterations = 500)

Wall time: 40min 48s


In [32]:
#Saving the model locally 
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/80s")
lda_model_80s.save(temp_file)

In [33]:
#Loading the model if it is already built
LDA = gensim.models.ldamodel.LdaModel
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/80s")
lda_model_80s = LDA.load(temp_file)

In [34]:
#investigating the word distribution over the different topics
#lda_model_80s.print_topics()

In [35]:
#import pyLDAvis
#import pyLDAvis.gensim_models as gensimvis
#pyLDAvis.enable_notebook()

In [36]:
vis80s = gensimvis.prepare(lda_model_80s,doc_term_matrix_80s,dictionary_80s,mds= 'mmds', R=30)
vis80s

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.070500 -0.001675       1        1  19.822625
0      0.000980 -0.180027       2        1  14.884730
1     -0.274307 -0.077352       3        1  12.022752
5     -0.124079  0.156446       4        1  10.916775
4      0.230834 -0.315972       5        1  10.916605
3      0.318832 -0.043897       6        1  10.082080
9      0.001413  0.392913       7        1   8.460503
8     -0.163521 -0.361021       8        1   5.118874
2     -0.387407  0.181278       9        1   4.045338
7      0.326754  0.249307      10        1   3.729718, topic_info=            Term         Freq        Total Category  logprob  loglift
32       country  4441.000000  4441.000000  Default  30.0000  30.0000
94       develop  1527.000000  1527.000000  Default  29.0000  29.0000
95      economic  1780.000000  1780.000000  Default  28.0000  28.0000
223      nuclear  1304.000000  1304.000000  Default  27.0000  27.0000
152  development  1689.000000  1689.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
333    community    57.883837  1262.634269  Topic10  -5.2202   0.2063
683      conduct    46.710974   156.961593  Topic10  -5.4347   2.0768
25       welcome    49.110470   366.455227  Topic10  -5.3846   1.2790
56   opportunity    48.727620   369.292354  Topic10  -5.3924   1.2635
678        shall    47.359597   281.231473  Topic10  -5.4209   1.5074

[636 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
749       2  0.809439     ability
749      10  0.181896     ability
448       5  0.977330      access
2426      3  0.988982  accomplish
478       1  0.320005     achieve
...     ...       ...         ...
1530      4  0.062252         yet
1530      6  0.071145         yet
1530      7  0.412046         yet
1530      8  0.100788         yet
2442      8  0.962507       young

[1141 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 1, 2, 6, 5, 4, 10, 9, 3, 8])

### 90s

In [37]:
%%time
# Placing the processed corpus in necessary format for LDA

dictionary_90s = corpora.Dictionary(processed_corpus_90s)
doc_term_matrix_90s = [dictionary_90s.doc2bow(speech) for speech in processed_corpus_90s]

# Creating the LDA object 

LDA = gensim.models.ldamodel.LdaModel

#building the model
lda_model_90s = LDA(corpus = doc_term_matrix_90s, id2word = dictionary_90s, num_topics = 10, random_state=100,
               chunksize= 1000, passes = 1000, iterations = 500)

Wall time: 32min 1s


In [38]:
#Saving the model locally 
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/90s")
lda_model_90s.save(temp_file)

In [39]:
#Loading the model if it is already built
LDA = gensim.models.ldamodel.LdaModel
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/90s")
lda_model_90s = LDA.load(temp_file)

In [40]:
#investigating the word distribution over the different topics
#lda_model_90s.print_topics()

In [41]:
#import pyLDAvis
#import pyLDAvis.gensim_models as gensimvis
#pyLDAvis.enable_notebook()

In [42]:
vis90s = gensimvis.prepare(lda_model_90s,doc_term_matrix_90s,dictionary_90s,mds= 'mmds', R=30)
vis90s

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.288954  0.115664       1        1  26.030677
7     -0.052114  0.129445       2        1  17.694108
5     -0.214411  0.288751       3        1  14.291002
2      0.157196 -0.122517       4        1  13.429835
1      0.386533 -0.076590       5        1   7.393982
6      0.307168  0.174437       6        1   6.730171
8      0.107618  0.425281       7        1   4.310073
3      0.110171 -0.400767       8        1   3.913358
0     -0.378124 -0.204395       9        1   3.686140
9     -0.135082 -0.329308      10        1   2.520654, topic_info=             Term         Freq        Total Category  logprob  loglift
890       nuclear  1020.000000  1020.000000  Default  30.0000  30.0000
249         right  1187.000000  1187.000000  Default  29.0000  29.0000
347      economic  1272.000000  1272.000000  Default  28.0000  28.0000
727        weapon   694.000000   694.000000  Default  27.0000  27.0000
345   development  1623.000000  1623.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
106     community    98.870166  1015.159913  Topic10  -4.0188   1.3517
1487        speak    53.278357   199.034678  Topic10  -4.6370   2.3627
424        global    81.756883   815.271566  Topic10  -4.2088   1.3809
399       economy    46.439760   463.479400  Topic10  -4.7744   1.3800
1380       regime    31.193204   139.614237  Topic10  -5.1724   2.1820

[585 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
7956      8  0.986025    abdicate
1317      4  0.188222       abuse
1317      9  0.810803       abuse
3775     10  0.966820      accede
198       1  0.990309  accordance
...     ...       ...         ...
273       8  0.175104        work
273       9  0.166614        work
5914      2  0.982837      worker
2214      5  0.988496       wrong
5277      2  0.991112     yeltsin

[874 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 8, 6, 3, 2, 7, 9, 4, 1, 10])

### 2000s

In [43]:
%%time
# Placing the processed corpus in necessary format for LDA

dictionary_2000s = corpora.Dictionary(processed_corpus_2000s)
doc_term_matrix_2000s = [dictionary_2000s.doc2bow(speech) for speech in processed_corpus_2000s]

# Creating the LDA object 

LDA = gensim.models.ldamodel.LdaModel

#building the model
lda_model_2000s = LDA(corpus = doc_term_matrix_2000s, id2word = dictionary_2000s, num_topics = 10, random_state=100,
               chunksize= 1000, passes = 1000, iterations = 500)

Wall time: 30min 15s


In [44]:
#Saving the model locally 
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/2000s")
lda_model_2000s.save(temp_file)

In [45]:
#Loading the model if it is already built
LDA = gensim.models.ldamodel.LdaModel
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/2000s")
lda_model_2000s = LDA.load(temp_file)

In [46]:
#investigating the word distribution over the different topics
#lda_model_2000s.print_topics()

In [47]:
#import pyLDAvis
#import pyLDAvis.gensim_models as gensimvis
#pyLDAvis.enable_notebook()

In [48]:
vis2000s = gensimvis.prepare(lda_model_2000s,doc_term_matrix_2000s,dictionary_2000s,mds= 'mmds', R=30)
vis2000s

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.040580 -0.000840       1        1  23.485761
5      0.048554 -0.298776       2        1  16.343534
7     -0.236475 -0.068951       3        1  13.386920
8     -0.133273  0.230454       4        1  11.978344
4      0.223083 -0.244004       5        1  10.830902
2      0.345114 -0.022821       6        1   6.514984
9      0.041007  0.340726       7        1   5.929755
0     -0.242883 -0.318075       8        1   5.308419
1     -0.382221  0.130843       9        1   3.540667
3      0.296514  0.251443      10        1   2.680714, topic_info=             Term         Freq        Total Category  logprob  loglift
250     terrorism   614.000000   614.000000  Default  30.0000  30.0000
102         right   746.000000   746.000000  Default  29.0000  29.0000
179       nuclear   540.000000   540.000000  Default  28.0000  28.0000
89          human   698.000000   698.000000  Default  27.0000  27.0000
62    development  1150.000000  1150.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
214        affair    19.171431    67.330142  Topic10  -5.2697   2.3629
932           day    21.122339   129.171609  Topic10  -5.1728   1.8083
17           last    20.981053   290.871515  Topic10  -5.1796   0.9898
1617         hard    17.613423    58.143768  Topic10  -5.3545   2.4248
42           high    17.963474   173.305449  Topic10  -5.3348   1.3524

[616 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
5014      3  0.979251                
3490     10  0.958368  accountability
268       1  0.472780         achieve
268       2  0.236390         achieve
268       3  0.022513         achieve
...     ...       ...             ...
402       8  0.170102             yet
402       9  0.208761             yet
1389      2  0.367301           young
1389     10  0.629658           young
175       8  0.940436            zone

[1058 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 6, 8, 9, 5, 3, 10, 1, 2, 4])

### 2010s

In [49]:
%%time
# Placing the processed corpus in necessary format for LDA

dictionary_2010s = corpora.Dictionary(processed_corpus_2010s)
doc_term_matrix_2010s = [dictionary_2010s.doc2bow(speech) for speech in processed_corpus_2010s]

# Creating the LDA object 

LDA = gensim.models.ldamodel.LdaModel

#building the model
lda_model_2010s = LDA(corpus = doc_term_matrix_2010s, id2word = dictionary_2010s, num_topics = 10, random_state=100,
               chunksize= 1000, passes = 1000, iterations = 500)

Wall time: 26min 18s


In [50]:
#Saving the model locally 
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/2010s")
lda_model_2010s.save(temp_file)

In [51]:
#Loading the model if it is already built
LDA = gensim.models.ldamodel.LdaModel
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/2010s")
lda_model_2010s = LDA.load(temp_file)

In [52]:
#investigating the word distribution over the different topics
#lda_model_2010s.print_topics()

In [53]:
#import pyLDAvis
#import pyLDAvis.gensim_models as gensimvis
#pyLDAvis.enable_notebook()

In [54]:
vis2010s = gensimvis.prepare(lda_model_2010s,doc_term_matrix_2010s,dictionary_2010s,mds= 'mmds', R=30)
vis2010s

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.263510  0.113318       1        1  19.578507
7     -0.020103  0.105591       2        1  15.855789
3     -0.193637  0.274398       3        1  11.475850
6      0.203022 -0.152034       4        1  10.772989
8      0.333768 -0.017893       5        1   9.629667
1      0.261539  0.177874       6        1   9.453877
4      0.090833  0.347949       7        1   8.935577
0      0.103925 -0.348636       8        1   8.312338
9     -0.355679 -0.171203       9        1   3.182377
2     -0.160157 -0.329363      10        1   2.803029, topic_info=             Term         Freq        Total Category  logprob  loglift
369         right   776.000000   776.000000  Default  30.0000  30.0000
351         human   587.000000   587.000000  Default  29.0000  29.0000
603   development  1097.000000  1097.000000  Default  28.0000  28.0000
324      security  1232.000000  1232.000000  Default  27.0000  27.0000
496       council   551.000000   551.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
563     humankind    21.159626   212.744216  Topic10  -5.3709   1.2665
199         first    21.416776   368.470391  Topic10  -5.3589   0.7293
2307      current    20.007720   129.668762  Topic10  -5.4269   1.7056
53          share    20.072146   274.724142  Topic10  -5.4237   0.9580
212       problem    19.832793   289.746532  Topic10  -5.4357   0.8928

[662 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
3125      9  0.941110      abroad
3426      7  0.974360  accomplish
3363     10  0.979332     account
85        1  0.380592     achieve
85        2  0.373411     achieve
...     ...       ...         ...
3828      4  0.967855       wrong
1600      7  0.968607       yeman
1601      9  0.971688       yemen
3870      7  0.959289       youth
750       2  0.989784        zone

[1202 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 8, 4, 7, 9, 2, 5, 1, 10, 3])